<h1 align = "center"><b> Facebook Prophet </b></h1>


In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from prophet import Prophet

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [18]:
tabela_vendas = pd.read_csv('../Demanda Diária.csv', index_col = 0)
tabela_vendas = tabela_vendas.loc[tabela_vendas['loja'] == 1]
tabela_vendas = tabela_vendas.loc[tabela_vendas['produto'] == 1]
tabela_vendas = tabela_vendas.drop(['loja', 'produto'], axis = 1)

série_vendas = pd.Series(tabela_vendas['itens_vendidos'], index = tabela_vendas.index)

In [19]:
def Transformar_Série(série):
	série_transformada, lambda_encontrado = stats.boxcox(série)
	série_transformada = pd.Series(série_transformada, index = série.index)
	print(f'Lambda encontrado: {lambda_encontrado}')

	série_transformada = série_transformada.diff().dropna()
	return série_transformada, lambda_encontrado

série_transformada, lambda_encontrado = Transformar_Série(série_vendas)

Lambda encontrado: 0.576185039143375


<h2 align = "center"><b> Criando o Modelo </b></h2>

&emsp;&emsp; O prophet impõe que os dados estejam em um dataframe com duas colunas (ds e y). A coluna ds deve conter as datas em formato YYYY-MM-DD e a coluna y deve conter os valores que queremos prever.

In [20]:
tabela_prophet = pd.DataFrame({'ds': série_transformada.index, 'y': série_transformada.values})

tabela_prophet_treino = tabela_prophet.iloc[:-31] # Retirando o último mês do treino
tabela_prophet_teste = tabela_prophet.iloc[-31:] # Utilizando o último mês como teste

In [21]:
modelo = Prophet()
modelo.fit(tabela_prophet_treino)

22:03:03 - cmdstanpy - INFO - Chain [1] start processing
22:03:04 - cmdstanpy - INFO - Chain [1] done processing


In [71]:
future = modelo.make_future_dataframe(periods = 31, freq = 'D')
previsão = modelo.predict(future)
previsão = previsão.set_index('ds', drop = False)

In [26]:
def Inverter_Transformação(série_transformada, lambda_encontrado, valor_inicial):
    série_invertida = [stats.boxcox(valor_inicial, lambda_encontrado)]
    for i in range(0, len(série_transformada)):
        série_invertida.append(série_invertida[i] + série_transformada[i])
        if (i == 3): display(série_invertida)
    série_invertida = pd.Series(série_invertida[1:], index = série_transformada.index).dropna()

    if lambda_encontrado == 0: return np.exp(série_invertida)
    else: return (série_invertida * lambda_encontrado + 1) ** (1 / lambda_encontrado)

In [ ]:
resultado = Inverter_Transformação(previsão['yhat'], lambda_encontrado, série_vendas.iloc[0]).tail(31)
resultado.index = série_vendas.tail(31).index

resultado = pd.concat([resultado, série_vendas.tail(31)], axis = 1)
resultado.rename(columns={0: 'Previsão', 'itens_vendidos': 'Valor Real'}, inplace=True)

In [68]:
print(resultado)

             Previsão  Valor Real
data                             
2017-12-01  21.942213          19
2017-12-02  23.263175          16
2017-12-03  23.615094          31
2017-12-04  15.013922           7
2017-12-05  17.192013          20
2017-12-06  17.395763          17
2017-12-07  17.680158          18
2017-12-08  18.901850          17
2017-12-09  20.327479          27
2017-12-10  20.879557          20
2017-12-11  12.976453          20
2017-12-12  15.266522          13
2017-12-13  15.716027          17
2017-12-14  16.258118          14
2017-12-15  17.722507          16
2017-12-16  19.405336          15
2017-12-17  20.240018          22
2017-12-18  12.684929          19
2017-12-19  15.187012           7
2017-12-20  15.852722          16
2017-12-21  16.594144          12
2017-12-22  18.248247           6
2017-12-23  20.104376          18
2017-12-24  21.073316          19
2017-12-25  13.442104          13
2017-12-26  16.049607          16
2017-12-27  16.747418          14
2017-12-28  17

In [69]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

print('RMSE: ', mean_squared_error(resultado['Valor Real'], resultado['Previsão'], squared = False))
print('MAE: ', mean_absolute_error(resultado['Valor Real'], resultado['Previsão']))

RMSE:  4.668299767330705
MAE:  3.581771298440391


In [ ]:
modelo.plot_components(previsão)

<img src="./Gráficos/plot_components.png" alt="drawing" width="100%"/>